## 3. Working with Generics
Generics allow you to define the specification of the data type of programming elements in a class or a method, until it is actually used in the program. In other words, generics allow you to write a class or method that can work with any data type.

In [4]:
public class MyGenericArray<T> {
    private T[] array;
      
    public MyGenericArray(int size) {
        array = new T[size + 1];
    }

    public T getItem(int index) {
        return array[index];
    }
    
    public void setItem(int index, T value) {
        array[index] = value;
    }
}


class Tester {
    public static void Main() {
        int len = 5;

        //declaring an int array
        MyGenericArray<int> intArray = new MyGenericArray<int>(len);
         
        //setting values
        for (int c = 0; c < len; c++) {
            intArray.setItem(c, c*5);
        }
         
        //retrieving the values
        for (int c = 0; c < len; c++) {
            Console.Write(intArray.getItem(c) + " ");
        }
        Console.WriteLine();
         
        //declaring a character array
        MyGenericArray<char> charArray = new MyGenericArray<char>(len);
         
        //setting values
        for (int c = 0; c < 5; c++) {
            charArray.setItem(c, (char)(c+97));
        }
         
        //retrieving the values
        for (int c = 0; c< 5; c++) {
            Console.Write(charArray.getItem(c) + " ");
        }
        Console.WriteLine();
    }
}


Tester.Main();

0 5 10 15 20 
a b c d e 


Some articles and papers might lead you to believe that generics are useful only in the context of collections. That’s not true. There are many other ways to use generics. You can use them to create interfaces, event handlers, common algorithms, and more.

### Item 18: Always Define Constraints That Are Minimal and Sufficient

### Item 19: Specialize Generic Algorithms Using Runtime Type Checking

### Item 20: Implement Ordering Relations with IComparable<T> and IComparer<T>


In [10]:
public struct Customer : IComparable<Customer>, IComparable {
    public readonly string name;
    
    public Customer(string name) {
        this.name = name;
    }
    
    // IComparable<Customer> Members
    public int CompareTo(Customer other) => name.CompareTo(other.name);
    
    // IComparable Members
    int IComparable.CompareTo(object obj) {
        if (!(obj is Customer)) {
            throw new ArgumentException("Argument is not a Customer", "obj");
        }
        Customer otherCustomer = (Customer)obj;
        return this.CompareTo(otherCustomer);
    }
}


In [12]:
List<Customer> customers = new List<Customer>(); 
customers.Add(new Customer("Simon"));
customers.Add(new Customer("Jake"));
customers.Add(new Customer("James"));

customers.Sort();
foreach(var customer in customers) {
    Console.WriteLine(customer.name);
}

Jake
James
Simon


There’s just too much to dislike about the classic version of IComparable. You’ve got to check the runtime type of the argument. Incorrect code could legally call this method with anything as the argument to the CompareTo method. More so, proper arguments must be boxed and unboxed to provide the actual comparison. That’s an extra runtime expense for each compare.

Adding operators to this code...

In [13]:
public struct Customer : IComparable<Customer>, IComparable {
    public readonly string name;
    
    public Customer(string name) {
        this.name = name;
    }
    
    // IComparable<Customer> Members
    public int CompareTo(Customer other) => name.CompareTo(other.name);
    
    // IComparable Members
    int IComparable.CompareTo(object obj) {
        if (!(obj is Customer)) {
            throw new ArgumentException("Argument is not a Customer", "obj");
        }
        Customer otherCustomer = (Customer)obj;
        return this.CompareTo(otherCustomer);
    }
    
    public static bool operator < (Customer left, Customer right) => left.CompareTo(right) < 0;
    
    public static bool operator <= (Customer left, Customer right) => left.CompareTo(right) <= 0;
    
    public static bool operator > (Customer left, Customer right) => left.CompareTo(right) > 0;
    
    public static bool operator >= (Customer left, Customer right) => left.CompareTo(right) >= 0;
}

In [14]:
var simon = new Customer("Simon");
var james = new Customer("James");

In [18]:
simon > james

True

In [19]:
simon >= james

True

In [16]:
simon < james

False

In [20]:
simon <= james

False

### Item 21: Always Create Generic Classes That Support Disposable Type Parameters

In [21]:
public interface IEngine {
    void DoWork();
}

public class EngineDriverOne<T> where T: IEngine, new() {
    public void GetThingsDone() {
        T driver = new T();
        driver.DoWork();
    }
}

You may have introduced a resource leak if T implements IDisposable. In every case where you create a local variable of type T, you need to check whether T implements IDisposable and, if so, dispose of it correctly.

In [22]:
public class EngineDriverOne<T> where T: IEngine, new() {
    public void GetThingsDone() {
        T driver = new T();
        using (driver as IDisposable) {
            driver.DoWork();
        }
    }
}

If T does not implement IDisposable, then the value of this local variable is null. In those cases, the compiler does not call Dispose(), because it checks against null before doing this extra work.

### Item 22: Support Generic Covariance and Contravariance

### Item 23: Use Delegates to Define Method Constraints on Type Parameters

In [1]:
public static class Example {
    public static T Add<T> (T left, T right, Func<T,T,T> AddFunc) => AddFunc(left, right);
}

In [3]:
int a = 6;
int b = 7;
int sum = Example.Add(a, b, (x, y) => x + y);
sum

13

### Item 24: Do Not Create Generic Specialization on Base Classes or Interfaces
You must pay careful attention to overload resolution, and you must determine when generic methods will create better matches than the methods developers might reasonably expect.

In [8]:
using System;


public class MyBase {}


public interface IMessageWriter {
    void WriteMessage();
}


public class MyDerived: MyBase, IMessageWriter {
    void IMessageWriter.WriteMessage() => Console.WriteLine("Inside MyDerived.WriteMessage");
}


public class AnotherType: IMessageWriter {
    public void WriteMessage() => Console.WriteLine("Inside AnotherType.WriteMessage");
}


class Program {
    static void WriteMessage(MyBase b) {
        Console.WriteLine("Inside WriteMessage(MyBase)");
    }
    
    static void WriteMessage <T>(T obj) {
        Console.Write("Inside WriteMessage<T>(T):");
    }
    
    static void WriteMessage(IMessageWriter obj) {
        Console.Write("Inside WriteMessage(IMessageWriter):");
        obj.WriteMessage();
    }
    
    public static void Main() {
        MyDerived d = new MyDerived();
        Console.WriteLine("Calling Program.WriteMessage");
        WriteMessage(d);
        Console.WriteLine();
        
        Console.WriteLine("Calling through IMessageWriter interface");
        WriteMessage((IMessageWriter)d);
        Console.WriteLine();
        
        Console.WriteLine("Cast to base object");
        WriteMessage((MyBase)d);
        Console.WriteLine();
        
        Console.WriteLine("Another Type test:");
        AnotherType anObject = new AnotherType();
        WriteMessage(anObject);
        Console.WriteLine();
        
        Console.WriteLine("Cast to IMessageWriter:");
        WriteMessage((IMessageWriter)anObject);
    }
}


Program.Main();

Calling Program.WriteMessage
Inside WriteMessage<T>(T):
Calling through IMessageWriter interface
Inside WriteMessage(IMessageWriter):Inside MyDerived.WriteMessage

Cast to base object
Inside WriteMessage(MyBase)

Another Type test:
Inside WriteMessage<T>(T):
Cast to IMessageWriter:
Inside WriteMessage(IMessageWriter):Inside AnotherType.WriteMessage


rules. Generics are almost always a good match, and they wreak havoc with our assumptions about which methods get called.

The first test shows one of the more important concepts to remember: WriteMessage<T>(T obj) is a better match than WriteMessage(MyBase b) for an expression whose type is derived from MyBase. MyBase. That’s because the compiler can make an exact match by substituting MyDerived for T in that message, and WriteMessage(MyBase) requires an implicit conversion. Generic methods are always perfect matches, so they win over base class methods.
    
The next two tests show how you can control this behavior by explicitly invoking the conversion (either to MyBase or to an IMessageWriter type).

### Item 25: Prefer Generic Methods Unless Type Parameters Are Instance Fields

In [9]:
public static class Utils {
    public static T Max<T>(T left, T right) =>
        Comparer<T>.Default.Compare(left, right) < 0 ? right : left;

    public static double Max(double left, double right) =>
        Math.Max(left, right);

    public static T Min<T>(T left, T right) =>
        Comparer<T>.Default.Compare(left, right) < 0 ? left: right;
        
    public static double Min(double left, double right) =>
        Math.Min(left, right);
}

In [11]:
double d1 = 4;
double d2 = 5;
double max = Utils.Max(d1, d2);
max

5

In [12]:
string foo = "foo";
string bar = "bar";
string sMax = Utils.Max(foo, bar);
sMax

foo

In [14]:
double? d3 = 12;
double? d4 = null;
double? Max2 = Utils.Max(d3, d4).Value;
Max2

12

If there is a specific version of the parameter type, the compiler calls that version. If there isn’t a specific version, the compiler calls the generic version.

### Item 26: Implement Classic Interfaces in Addition to Generic Interfaces

In [2]:
class Name : IComparable<Name>, IEquatable<Name> {
    public string First {get; set;}
    public string Last {get; set;}
    public string Middle {get; set;}
    
    public Name(string First, string Middle, string Last) {
        this.First = First;
        this.Middle = Middle;
        this.Last = Last;
    }
    
    public int CompareTo(Name other) {
        if (Object.ReferenceEquals(this, other)) {
            return 0;
        }
        if (Object.ReferenceEquals(other, null)) {
            return 1; // any non-null object > null
        }
        
        int rVal = Comparer<string>.Default.Compare(Last, other.Last);
        if (rVal != 0) {
            return rVal;
        } 
        
        rVal = Comparer<string>.Default.Compare(First, other.First);
        if (rVal != 0) {
            return rVal;
        }
        
        return Comparer<string>.Default.Compare(Middle, other.Middle);
    }
    
    public bool Equals(Name other) {
        if (Object.ReferenceEquals(this, other)) {
            return true;
        }
        if (Object.ReferenceEquals(other, null)) {
            return false;
        }
        
        // Symantically equivalent to using EqualityComparer<string>.Default
        return Last == other.Last && First == other.First && Middle == other.Middle;
    }
}


In [4]:
var simon = new Name("Simon", "Smithy", "Smith");
var james = new Name("James", "Smithy", "Smith");

In [9]:
simon.CompareTo(james)

1

In [10]:
james.CompareTo(simon)

-1

In [11]:
james.CompareTo(james)

0

### Item 27: Augment Minimal Interface Contracts with Extension Methods

In [19]:
public interface IFoo {
    int Marker {get; set;}
}

/*
public static class FooExtensions {
    public static void NextMarker(this IFoo thing) {
        thing.Marker += 1;
    }
}
*/

public static void NextMarker(this IFoo thing) => thing.Marker += 1;


public class MyType : IFoo {
    public int Marker {get; set;}
    
    public void NextMarker() => Marker += 5;
}


MyType t = new MyType();
t.Marker

0

In [22]:
NextMarker(t);
t.Marker

1

In [26]:
public class MyType2 : IFoo {
    public int Marker {get; set;}
    public void NextMarker() => Marker += 5;
}

In [28]:
MyType2 t2 = new MyType2();
t2.Marker

0

In [29]:
NextMarker(t2);
t2.Marker

1

In [30]:
t2.NextMarker();
t2.Marker

6

### Item 28: Consider Enhancing Constructed Types with Extension Methods

***